## Imports

In [ ]:
import sys
from datetime import datetime
import os

import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
import pandas as pd
import torchvision
import torchvision.transforms as transforms
from torch import optim

from torchvision import models

from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, cohen_kappa_score, \
    top_k_accuracy_score, confusion_matrix, classification_report

import plotly.express as px

## Download the dataset

In [ ]:
from google.colab import userdata
import os

os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')

In [ ]:
!kaggle datasets download -d gpiosenka/cards-image-datasetclassification

! unzip "cards-image-datasetclassification.zip" -d "data"

Streaming output truncated to the last 5000 lines.
  inflating: data/train/jack of spades/059.jpg  
  inflating: data/train/jack of spades/060.jpg  
  inflating: data/train/jack of spades/061.jpg  
  inflating: data/train/jack of spades/062.jpg  
  inflating: data/train/jack of spades/063.jpg  
  inflating: data/train/jack of spades/064.jpg  
  inflating: data/train/jack of spades/065.jpg  
  inflating: data/train/jack of spades/066.jpg  
  inflating: data/train/jack of spades/067.jpg  
  inflating: data/train/jack of spades/068.jpg  
  inflating: data/train/jack of spades/069.jpg  
  inflating: data/train/jack of spades/070.jpg  
  inflating: data/train/jack of spades/071.jpg  
  inflating: data/train/jack of spades/072.jpg  
  inflating: data/train/jack of spades/073.jpg  
  inflating: data/train/jack of spades/074.jpg  
  inflating: data/train/jack of spades/075.jpg  
  inflating: data/train/jack of spades/076.jpg  
  inflating: data/train/jack of spades/077.jpg  
  inflating: data/

## CNN Architecture

In [ ]:
class CNNClassifier(nn.Module):
    def __init__(self, in_channel, output_dim):
        super(CNNClassifier, self).__init__()

        self.model = nn.Sequential(
            nn.Conv2d(in_channel, 64, 3),
            nn.ReLU(),
            nn.BatchNorm2d(64),

            nn.Conv2d(64, 64, 3, 2),
            nn.ReLU(),
            nn.BatchNorm2d(64),

            nn.Conv2d(64, 128, 3),
            nn.ReLU(),
            nn.BatchNorm2d(128),

            nn.Conv2d(128, 128, 3, 2),
            nn.ReLU(),
            nn.BatchNorm2d(128),

            nn.Conv2d(128, 256, 3),
            nn.ReLU(),
            nn.BatchNorm2d(256),

            nn.Conv2d(256, 256, 3, 2),
            nn.ReLU(),
            nn.BatchNorm2d(256),

            nn.Conv2d(256, 512, 3),
            nn.ReLU(),
            nn.BatchNorm2d(512),

            nn.Conv2d(512, 512, 3, 2),
            nn.ReLU(),
            nn.BatchNorm2d(512),

            nn.Flatten(),

            nn.Linear(73728, 120),
            nn.ReLU(),

            nn.Linear(120, 84),
            nn.ReLU(),

            nn.Linear(84, output_dim),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        return self.model(x)


## Functions

In [ ]:
def print_txt(text: str, txt_file):
    print(text)
    txt_file.write(text)
    txt_file.write("\n")

In [ ]:
def get_model_information(model, txt_file):
  # Print the model architecture
  print_txt(str(model), txt_file)

  # Calculate the number of parameters
  total_params = sum(p.numel() for p in model.parameters())
  trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

  print_txt(f"Total parameters: {total_params}", txt_file)
  print_txt(f"Trainable parameters: {trainable_params}", txt_file)

In [ ]:
# Transform a one hot vector into an integer (index of the maximum value)
def vec_to_int(y_true, y_predicted):
    # Vérifiez si y_true est déjà un vecteur d'indices
    if len(y_true.shape) == 1:
        y_true_int = y_true
    else:
        y_true_int = np.argmax(y_true, axis=1)

    # Vérifiez si y_predicted est déjà un vecteur d'indices
    if len(y_predicted.shape) == 1:
        y_predicted_int = y_predicted
    else:
        y_predicted_int = np.argmax(y_predicted, axis=1)

    return y_true_int, y_predicted_int

In [ ]:
# This function can be used for the training and validation stage
def model_performances(y_true, y_predicted, loss, my_score_df):

    # Initialize a list
    scores = []

    # Get the index of the maximum value in the vectors "y_true" and "y_predicted"
    y_int_true, y_int_predicted = vec_to_int(y_true, y_predicted)

    # Compute scores and add them to the list scores
    scores.extend([loss])
    scores.extend([accuracy_score(y_int_true, y_int_predicted)])
    scores.extend([balanced_accuracy_score(y_int_true, y_int_predicted)])
    scores.extend([f1_score(y_int_true, y_int_predicted, average="micro")])
    scores.extend([cohen_kappa_score(y_int_true, y_int_predicted)])
    scores.extend([top_k_accuracy_score(y_int_true, y_predicted, k=2)])
    scores.extend([top_k_accuracy_score(y_int_true, y_predicted, k=3)])

    my_score_df.loc[len(my_score_df)] = scores

    return my_score_df

In [ ]:
# This function is only to use on the test set
def show_compute_model_performances(y_true, y_predicted, loss, my_score_df, classes, txt_file):

    # Initialize a list
    scores = []

    # Get the index of the maximum value in the vectors "y_true" and "y_predicted"
    y_int_true, y_int_predicted = vec_to_int(y_true, y_predicted)

    # Compute scores, add them to the list scores and show them
    scores.extend([loss])

    # Accuracy
    accuracy = accuracy_score(y_int_true, y_int_predicted)
    print_txt(f"Accuracy: {accuracy}", txt_file)
    scores.extend([accuracy])

    # Balanced accuracy
    balanced_accuracy = balanced_accuracy_score(y_int_true, y_int_predicted)
    print_txt(f"Balanced Accuracy: {balanced_accuracy}", txt_file)
    scores.extend([balanced_accuracy])

    # F1-score
    f1 = f1_score(y_int_true, y_int_predicted, average="micro")
    print_txt(f"F1-score: {f1}", txt_file)
    scores.extend([f1])

    # Cohen Kappa
    kappa = cohen_kappa_score(y_int_true, y_int_predicted)
    print_txt(f"Kappa: {kappa}", txt_file)
    scores.extend([kappa])

    # Confusion matrix
    if len(y_true.shape) > 1 and y_true.shape[1] <= 10:  # Vérifie si y_true est 2D
        print_txt(str(confusion_matrix(y_int_true, y_int_predicted)), txt_file)

    # Classification report
    print_txt(str(classification_report(y_int_true, y_int_predicted, target_names=classes)), txt_file)

    # Top 2 accuracy
    top_2 = top_k_accuracy_score(y_int_true, y_predicted, k=2)
    print_txt(f"Top 2 Accuracy: {top_2}", txt_file)
    scores.extend([top_2])

    # Top 3 accuracy
    top_3 = top_k_accuracy_score(y_int_true, y_predicted, k=3)
    print_txt(f"Top 3 Accuracy: {top_3}", txt_file)
    scores.extend([top_3])

    my_score_df.loc[len(my_score_df)] = scores

    return my_score_df

In [ ]:
def create_score_df(training_epoch_scores, validation_epoch_scores, score_type):

    # Create a DataFrame for plotting the train "score_type"
    train_df = pd.DataFrame(columns=["Epochs", "Stage", score_type])
    # Create the vectors of values for the epochs and the stage of the training process
    epochs = np.arange(1, training_epoch_scores.shape[0] + 1, 1)
    stage = ["Train"] * training_epoch_scores.shape[0]
    # Fill the DataFrame for the train "score_type"
    train_df["Epochs"] = epochs
    train_df["Stage"] = stage
    train_df[score_type] = training_epoch_scores[score_type]

    # Create a DataFrame for plotting the validation "score_type"
    validation_df = pd.DataFrame(columns=["Epochs", "Stage", score_type])
    # Create the vector of values for the stage of the training process
    stage = ["Validation"] * training_epoch_scores.shape[0]
    # Fill the DataFrame for the validation "score_type"
    validation_df["Epochs"] = epochs
    validation_df["Stage"] = stage
    validation_df[score_type] = validation_epoch_scores[score_type]

    # Merge the two DataFrame
    score_df = pd.concat([train_df, validation_df])

    return score_df

In [ ]:
def plot_score_graphs(training_epoch_scores, validation_epoch_scores, results_path, my_folder_name):

    # List of scores to plot
    scores_to_plot = ["Loss", "Accuracy", "Balanced Accuracy", "F1-score", "Kappa", "Top 2 Accuracy", "Top 3 Accuracy"]

    # For each score to plot in the defined list
    for score_type in scores_to_plot:

        # Create the DataFrame to be used with Plotly
        the_df = create_score_df(training_epoch_scores, validation_epoch_scores, score_type)

        # Plot the score evolution for the training and validation stages
        fig = px.line(the_df, x="Epochs", y=score_type, color="Stage")
        # Temporary path
        temp_path = os.path.join(results_path, my_folder_name)
        # Save the graph
        fig.write_html(os.path.join(temp_path, score_type + ".html"))

In [ ]:
def compute_accuracy(labels, outputs):

    # Ajustement des dimensions si nécessaire
    if len(labels.shape) > 1:  # One-hot encoded labels
        labels = labels.argmax(dim=1)
    outputs = outputs.argmax(dim=1)

    # Calcul de l'exactitude
    accuracy = (labels == outputs).float().mean().item()
    return accuracy

In [ ]:
def compute_model_outputs(inputs, labels, device, model, all_labels, all_outputs, loss_function):
    # Déplacer les entrées et les labels sur le bon appareil
    inputs, labels = inputs.to(device), labels.to(device)

    # Calculer les sorties du modèle
    outputs = model(inputs)

    # Si les labels sont en one-hot, convertissez-les en indices de classe
    if labels.ndimension() > 1:  # vérifier si les labels sont one-hot encodés
        labels = torch.argmax(labels, dim=1)  # convertissez en indices de classe

    # Calculer la perte
    loss = loss_function(outputs, labels)

    # Calculer la précision
    accuracy = compute_accuracy(labels, outputs)

    # Ajouter les résultats à la liste
    all_labels.extend(labels.cpu().numpy())
    all_outputs.extend(outputs.cpu().detach().numpy())

    return all_labels, all_outputs, loss, accuracy

## Functions : Train, Validate, Test

In [ ]:
def test_model(test_loader, model, loss_function, device, classes, txt_file):

    # Initialize a DataFrame where to store metrics
    test_scores = pd.DataFrame(columns=["Loss", "Accuracy", "Balanced Accuracy", "F1-score", "Kappa",
                                        "Top 2 Accuracy", "Top 3 Accuracy"])

    # Tell to your model that your are evaluating it
    model.eval()

    # Initialize a mini-batch counter
    mini_batch_counter = 0

    # Initialize the loss and accuracy
    running_loss = 0.0
    running_accuracy = 0.0

    # Initialize two variables to store the outputs of the neural network and the labels (for the whole test set)
    all_outputs = []
    all_labels = []

    # Assign the tqdm iterator to the variable "progress_testing"
    with tqdm(test_loader, unit=" mini-batch") as progress_testing:

        # For each mini-batch defined in the validation loader through the variable "progress_validation"
        for inputs, labels in progress_testing:

            progress_testing.set_description("Testing the training model")

            # Compute the outputs of the model with specific inputs
            all_labels, all_outputs, loss, accuracy = compute_model_outputs(inputs, labels, device, model,
                                                                            all_labels, all_outputs, loss_function)

            # Update the running loss
            running_loss += loss.item()
            # Update the running accuracy
            running_accuracy += accuracy

            # Display the updated loss and the accuracy
            progress_testing.set_postfix(testing_loss=running_loss / (mini_batch_counter + 1),
                                         testing_accuracy=100. * (running_accuracy / (mini_batch_counter + 1)))

            # Increment the mini-batch counter
            mini_batch_counter += 1

    # Compute the performances on the test set and store them
    test_scores = show_compute_model_performances(np.array(all_labels), np.array(all_outputs),
                                                  running_loss / mini_batch_counter, test_scores, classes, txt_file)

    return test_scores

In [ ]:
def train_model(epoch_number, train_loader, validation_loader, model, optimizer, loss_function, device, results_path, my_folder_name):
    # Initialize a DataFrame where to store metrics
    training_epoch_scores = pd.DataFrame(columns=["Loss", "Accuracy", "Balanced Accuracy", "F1-score", "Kappa",
                                                  "Top 2 Accuracy", "Top 3 Accuracy"])
    validation_epoch_scores = pd.DataFrame(columns=["Loss", "Accuracy", "Balanced Accuracy", "F1-score", "Kappa",
                                                    "Top 2 Accuracy", "Top 3 Accuracy"])

    # Tell to your model that your are training it
    model.train()

    # For each epoch
    for epoch in range(epoch_number):

        # Initialize a mini-batch counter
        mini_batch_counter = 0

        # Initialize the loss and accuracy
        running_loss = 0.0
        running_accuracy = 0.0

        # Initialize two variables to store the outputs of the neural network and the labels (for the whole epoch)
        all_outputs = []
        all_labels = []

        # Assign the tqdm iterator to the variable "progress_epoch"
        with tqdm(train_loader, unit=" mini-batch") as progress_epoch:

            # For each mini-batch defined in the train loader through the variable "progress_epoch"
            for inputs, labels in progress_epoch:

                # Set the description of the progress bar
                progress_epoch.set_description(f"Epoch {epoch + 1}/{epoch_number}")

                # Compute the outputs of the model with specific inputs
                all_labels, all_outputs, loss, accuracy = compute_model_outputs(inputs, labels, device, model,
                                                                                all_labels, all_outputs, loss_function)

                # Update the weights and biais of the network
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                # Update the running loss
                running_loss += loss.item()
                # Update the running accuracy
                running_accuracy += accuracy

                # Display the updated loss and the accuracy
                progress_epoch.set_postfix(train_loss=running_loss / (mini_batch_counter + 1),
                                           train_accuracy=100. * (running_accuracy / (mini_batch_counter + 1)))

                # Increment the mini-batch counter
                mini_batch_counter += 1

        # Compute the performances of the training on the current epoch and store the scores
        training_epoch_scores = model_performances(np.array(all_labels), np.array(all_outputs),
                                                   running_loss / mini_batch_counter, training_epoch_scores)

        # Check performance of the model on the validation set after each training epoch
        validation_epoch_scores = validate_model(validation_loader, model, loss_function, device,
                                                 validation_epoch_scores)

    # Plot metrics
    plot_score_graphs(training_epoch_scores, validation_epoch_scores, results_path, my_folder_name)

    return model

In [ ]:
def validate_model(validation_loader, model, loss_function, device, validation_epoch_scores):

    # Tell to your model that your are evaluating it
    model.eval()

    # Initialize a mini-batch counter
    mini_batch_counter = 0

    # Initialize the loss and accuracy
    running_loss = 0.0
    running_accuracy = 0.0

    # Initialize two variables to store the outputs of the neural network and the labels (for the whole validation set
    # at the end of the current epoch)
    all_outputs = []
    all_labels = []

    # Assign the tqdm iterator to the variable "progress_validation"
    with tqdm(validation_loader, unit=" mini-batch") as progress_validation:

        # For each mini-batch defined in the validation loader through the variable "progress_validation"
        for inputs, labels in progress_validation:

            # Set the description of the progress bar
            progress_validation.set_description("               Validation step")

            # Compute the outputs of the model with specific inputs
            all_labels, all_outputs, loss, accuracy = compute_model_outputs(inputs, labels, device, model, all_labels,
                                                                            all_outputs, loss_function)

            # Update the running loss
            running_loss += loss.item()
            # Update the running accuracy
            running_accuracy += accuracy

            # Display the updated loss and the accuracy
            progress_validation.set_postfix(validation_loss=running_loss / (mini_batch_counter + 1),
                                            validation_accuracy=100. * (running_accuracy / (mini_batch_counter + 1)))

            # Increment the mini-batch counter
            mini_batch_counter += 1

    # Compute the performances on the validation set of the current epoch and store the scores
    validation_epoch_scores = model_performances(np.array(all_labels), np.array(all_outputs),
                                                 running_loss / mini_batch_counter, validation_epoch_scores)

    return validation_epoch_scores

## Paramètres de test

In [207]:
# model_name = "CNN"
model_name = "VGG"

dataset_path = "data"

results_path = "./results"
results_path = results_path + "_" + model_name

# Define the number of epochs of the model training
epoch_number = 20

# Define the size of the mini-batch
batch_size = 64

# Define the learning rate
learning_rate = 0.005

weight_decay = 0.001

## Create logs

In [208]:
# Get the date and time
now = datetime.now()
# Create the folder name
my_folder_name = now.strftime("%Y-%m-%d_%H" + "h" + "%M" + "min" + "%S" + "sec")
# Create the folder
os.makedirs(os.path.join(results_path, my_folder_name))
# Print a message in the console
print("\nResult folder created")

# Create and open a txt file to store information about the model performances
txt_file = open(os.path.join(os.path.join(results_path, my_folder_name), "Results.txt"), "a")

# Write information about hyperparameters
print_txt("Hyperparameters", txt_file)
print_txt(f"Model name: {model_name}", txt_file)
print_txt(f"Epoch number: {epoch_number}", txt_file)
print_txt(f"Batch size: {batch_size}", txt_file)
print_txt(f"Learning rate: {learning_rate}", txt_file)
print_txt(f"Weight decay: {weight_decay}", txt_file)



Result folder created
Hyperparameters
Model name: VGG
Epoch number: 20
Batch size: 64
Learning rate: 0.005
Weight decay: 0.001


## Load dataset  

In [209]:
""" Data Transformation """

if model_name == "CNN":
    transform = transforms.Compose([
        transforms.Resize((240, 240)),
        transforms.ToTensor(),
    ])
elif model_name == "VGG":
    transform = transforms.Compose([
        transforms.Resize((256,256)),
        transforms.ToTensor(),
    ])

target_transform = transforms.Compose([
    transforms.Lambda(
        lambda y: torch.zeros(53, dtype=torch.float).scatter_(0, torch.tensor(y), value=1)
    )
])

In [210]:
""" Load dataset """

train_dataset = torchvision.datasets.ImageFolder(
    dataset_path + "/train",
    transform=transform,
    target_transform=target_transform
)

validation_dataset = torchvision.datasets.ImageFolder(
    dataset_path + "/valid",
    transform=transform,
    target_transform=target_transform
)

test_dataset = torchvision.datasets.ImageFolder(
    dataset_path + "/test",
    transform=transform,
    target_transform=target_transform
)

In [211]:
""" Create data loader """

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True
)

validation_loader = torch.utils.data.DataLoader(
    validation_dataset,
    batch_size=batch_size,
    shuffle=False
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False
)

In [212]:
""" Get data informations """

# Load the first batch of images from the train set
images, labels = next(iter(train_loader))

# Get the shape of the images
image_shape = list(images.data.shape)
# Get automatically the number of channels of images
image_channel = image_shape[1]

# Get the number of classes from the dataset
classes = train_dataset.classes
class_number = len(list(classes))

## Création du model

In [213]:
# Check if GPU is available and set the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Instantiate the model
if model_name == "CNN":
    model = CNNClassifier(in_channel=image_channel, output_dim=class_number)

elif model_name == "VGG":
    model = models.vgg19(weights=models.VGG19_Weights.IMAGENET1K_V1)

    # Geler tous les poids de vgg
    for param in model.parameters():
        param.requires_grad = False

    model.classifier[6] = nn.Linear(4096, class_number)

# move the model to GPU
model = model.to(device)

# Print information about the model
get_model_information(model, txt_file)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

## Optimizer and loss function

In [214]:
# Create the loss function
loss_function = nn.CrossEntropyLoss()
# Create the optimizer
optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

## Training

In [215]:
# Train the neural network
model = train_model(
    epoch_number,
    train_loader,
    validation_loader,
    model, optimizer,
    loss_function,
    device,
    results_path,
    my_folder_name
)

               Validation step: 100%|██████████| 5/5 [00:02<00:00,  1.84 mini-batch/s, validation_accuracy=56.1, validation_loss=1.86]


In [216]:
# Test the neural network
test_model(
    test_loader,
    model,
    loss_function,
    device,
    classes,
    txt_file
)

Testing the training model: 100%|██████████| 5/5 [00:02<00:00,  1.85 mini-batch/s, testing_accuracy=47.9, testing_loss=3.17]

Accuracy: 0.5320754716981132
Balanced Accuracy: 0.5320754716981132
F1-score: 0.5320754716981132
Kappa: 0.523076923076923
                   precision    recall  f1-score   support

     ace of clubs       0.60      0.60      0.60         5
  ace of diamonds       0.83      1.00      0.91         5
    ace of hearts       1.00      1.00      1.00         5
    ace of spades       1.00      1.00      1.00         5
   eight of clubs       0.29      0.40      0.33         5
eight of diamonds       0.40      0.40      0.40         5
  eight of hearts       0.62      1.00      0.77         5
  eight of spades       0.00      0.00      0.00         5
    five of clubs       0.29      0.40      0.33         5
 five of diamonds       0.43      0.60      0.50         5
   five of hearts       0.40      0.40      0.40         5
   five of spades       0.50      0.20      0.29         5
    four of clubs       0.50      0.60      0.55         5
 four of diamonds       1.00      0.40      0.57    


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



,Loss,Accuracy,Balanced Accuracy,F1-score,Kappa,Top 2 Accuracy,Top 3 Accuracy
0,3.166297,0.532075,0.532075,0.532075,0.523077,0.649057,0.713208


In [217]:
# Close your txt file
txt_file.close()